<a href="https://colab.research.google.com/github/blkeyd/CVPR/blob/main/Final/Paper/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create file to save the DCGAN generated images

In [ ]:
import os
os.makedirs("/content/drive/MyDrive/gan_generated_fire", exist_ok=True)

Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


Paths

In [ ]:
fire_data_dir = "/content/drive/MyDrive/fire_dataset_split/train/fire"
output_dir = "/content/drive/MyDrive/gan_generated_fire"
os.makedirs(output_dir, exist_ok=True)

Hyperparameters

In [ ]:
image_size = 64
batch_size = 64
latent_dim = 100
epochs = 50
lr = 0.0002
beta1 = 0.5

Transform (DCGAN standard)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

Dataset and Loader (Only fire images)

In [ ]:
dataset = datasets.ImageFolder(
    root=os.path.dirname(fire_data_dir),
    transform=transform
)

# Force dataset to use only "fire" class
fire_class_index = dataset.class_to_idx['fire']
dataset.samples = [s for s in dataset.samples if s[1] == fire_class_index]

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print("Fire images used for GAN:", len(dataset))

Fire images used for GAN: 528


DCGAN generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)

DCGAN discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1, 1).squeeze(1)

Initialize models

In [ ]:
G = Generator().to(device)
D = Discriminator().to(device)

Loss and Optimizers

In [ ]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=(beta1, 0.999))

Training DCGAN

In [ ]:
print("Starting DCGAN training...")

for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = imgs.to(device)
        batch_size_curr = real_imgs.size(0)

        real_labels = torch.ones(batch_size_curr, device=device)
        fake_labels = torch.zeros(batch_size_curr, device=device)

        # ---- Train Discriminator ----
        optimizer_D.zero_grad()
        real_loss = criterion(D(real_imgs), real_labels)

        z = torch.randn(batch_size_curr, latent_dim, 1, 1, device=device)
        fake_imgs = G(z)
        fake_loss = criterion(D(fake_imgs.detach()), fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # ---- Train Generator ----
        optimizer_G.zero_grad()
        g_loss = criterion(D(fake_imgs), real_labels)
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{epochs}]  D Loss: {d_loss.item():.4f}  G Loss: {g_loss.item():.4f}")

Starting DCGAN training...
Epoch [1/50]  D Loss: 0.3142  G Loss: 8.6496
Epoch [2/50]  D Loss: 0.7625  G Loss: 14.5790
Epoch [3/50]  D Loss: 0.0080  G Loss: 13.5526
Epoch [4/50]  D Loss: 0.0066  G Loss: 8.1778
Epoch [5/50]  D Loss: 0.0038  G Loss: 9.7618
Epoch [6/50]  D Loss: 0.0175  G Loss: 6.1607
Epoch [7/50]  D Loss: 3.9493  G Loss: 15.9955
Epoch [8/50]  D Loss: 0.5651  G Loss: 13.8455
Epoch [9/50]  D Loss: 0.2199  G Loss: 6.7488
Epoch [10/50]  D Loss: 0.5260  G Loss: 10.2807
Epoch [11/50]  D Loss: 0.1959  G Loss: 5.3856
Epoch [12/50]  D Loss: 0.3025  G Loss: 4.3755
Epoch [13/50]  D Loss: 0.0503  G Loss: 3.9347
Epoch [14/50]  D Loss: 0.3715  G Loss: 5.5979
Epoch [15/50]  D Loss: 0.4583  G Loss: 3.9378
Epoch [16/50]  D Loss: 1.1129  G Loss: 5.5529
Epoch [17/50]  D Loss: 0.2031  G Loss: 3.5478
Epoch [18/50]  D Loss: 0.1779  G Loss: 1.6915
Epoch [19/50]  D Loss: 0.1451  G Loss: 3.7877
Epoch [20/50]  D Loss: 0.1091  G Loss: 3.2784
Epoch [21/50]  D Loss: 0.4898  G Loss: 4.9017
Epoch [22/5

Generate Synthetic Fire Images

In [ ]:
print("Generating synthetic fire images...")
G.eval()
num_generate = 500  # you can increase later

with torch.no_grad():
    for i in range(num_generate):
        z = torch.randn(1, latent_dim, 1, 1, device=device)
        fake_img = G(z)
        save_image(fake_img, f"{output_dir}/fire_fake_{i}.png", normalize=True)

print(f"{num_generate} synthetic fire images saved to {output_dir}")

Generating synthetic fire images...
500 synthetic fire images saved to /content/drive/MyDrive/gan_generated_fire


Save Generator Model

In [ ]:
torch.save(G.state_dict(), "/content/drive/MyDrive/dcgan_generator_fire.pth")
print("DCGAN Generator model saved.")

DCGAN Generator model saved.
